In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import re
import string
import collections
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.cluster import MiniBatchKMeans

from time import time
%matplotlib inline

import nltk
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))
porter=PorterStemmer()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
import spacy
import spacy.cli
from spacy.matcher import Matcher 
from spacy.matcher import PhraseMatcher

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
spacy.cli.download("en")
spacy.cli.download("en_core_web_lg")
nlp = spacy.load('en_core_web_lg')
sbd = nlp.create_pipe('sentencizer')
nlp.add_pipe(sbd)

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


**Data Cleaning functions**

In [0]:
def cleanEmptyData(columnName,df):
    return df[df[columnName].notnull()]


## Remove Punctuation----------
def remove_punctuation(columnName,df):
    return df.loc[:,columnName].apply(lambda x: re.sub('[^a-zA-z\s]','',x))

## Convert To Lower Case
def lower_case(input_str):
    input_str = input_str.lower()
    return input_str  

def removeDuplicate(df,list):
    df.drop_duplicates(list, inplace=True)    

def remove_stop_words(columnName,df):
  return df.loc[:,columnName].apply(lambda x: [word for word in x.split() if word not in stop_words])

def remove_one_character_word(columnName,df):
  return df.loc[:,columnName].apply(lambda x: [i for i in x if len(i) > 1])

def join_seperator(columnName,df):
  seperator = ', '
  return df.loc[:,columnName].apply(lambda x: seperator.join(x))

def apply_stemmer(columnName,df):
  return df.loc[:,columnName].apply(lambda x: [porter.stem(word) for word in x])

## Data Cleaning Process
def dataCleaningProcess(dataFrame):
    removeDuplicate(dataFrame,['abstract', 'text_body'])
    clean_data = cleanEmptyData('text_body',dataFrame)
    clean_data.loc[:,'text_body_clean'] = clean_data.loc[:,'text_body'].apply(lambda x: lower_case(x))
    print('Lowe case Conversion completed')
    clean_data.loc[:,'text_body_clean'] = remove_punctuation('text_body_clean',clean_data)
    print('Punctuation Remove completed')
    clean_data.loc[:,'text_body_clean'] = remove_stop_words('text_body_clean',clean_data)
    print('Remove Stop Words completed')
    clean_data.loc[:,'text_body_clean'] = apply_stemmer('text_body_clean',clean_data)
    print('Stemmer APplied completed')
    clean_data.loc[:,'text_body_clean'] = remove_one_character_word('text_body_clean',clean_data)
    print('Remove One Chater completed')
    clean_data.loc[:,'text_body_clean'] = join_seperator('text_body_clean',clean_data)
    print('Join to single string')
    clean_data.loc[:,'text_body_clean'] = remove_punctuation('text_body_clean',clean_data)
    print('RFemove coma from the sentence')
    return clean_data

**Helper Functions**

In [0]:
def getWordsFromText(_text):
    words = []
    for i in range(0,len(_text)):
        words.append(str(_text.iloc[i]['text_body']).split(" "))
    return words

# Read Excel data as Data Frame
def readExcelToDataFrame(path):
    research_dataframe = pd.read_csv(path,index_col=False)
    research_dataframe.drop(research_dataframe.columns[research_dataframe.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    return research_dataframe

## basic scatter plot
def showScatterPlot(_X,title):
    # sns settings
    sns.set(rc={'figure.figsize':(15,15)})
    # colors
    palette = sns.color_palette("bright", 1)
    # plot
    sns.scatterplot(_X[:,0], _X[:,1], palette=palette)
    plt.title(title)
    # plt.savefig("plots/t-sne_covid19.png")
    plt.show()

## scatter plot with cluster
def showClusterScatterPlot(_X, _y_pred, title):
    # sns settings
    sns.set(rc={'figure.figsize':(10,10)})
    # colors
    palette = sns.color_palette("bright", len(set(_y_pred)))
    # plot
    sns.scatterplot(_X[:,0], _X[:,1], hue=_y_pred, legend='full', palette=palette)
    plt.title(title)
    # plt.savefig("plots/t-sne_covid19_label.png")
    plt.show()


## drop clumns
def getTargetData(dataFrame):
    text_body = dataFrame.drop(["doc_id", "source", "title", "abstract"], axis=1)
    return getWordsFromText(text_body)

def trainEmbededData(_perplexity,dataFrame,total_cluster, _n_iter):
    vectorizer = TfidfVectorizer(max_features=2**12)
    print('TfidfVectorizer completed')
    _X = vectorizer.fit_transform(dataFrame['text_body_clean'].values)
    print('Fit & Fransform completed')
    tsne = TSNE(verbose=1, perplexity=_perplexity,learning_rate=200, random_state=0, n_iter=_n_iter)
    _X_embeded = tsne.fit_transform(_X.toarray())
    print('Embeded Fit & Fransform completed')
    _kmeans = MiniBatchKMeans(n_clusters=total_cluster)
    return _X_embeded,_kmeans,_X

def predict(_kmeans,_X):
    return _kmeans.fit_predict(_X)
    
def analyse(pplexity,data_frame,cluster,iter):
    embeded,kmeans,x = trainEmbededData(pplexity,data_frame,cluster,iter)
    print('Completed Training')
    pred = predict(kmeans,x)
    print('Prediction completed')
    showClusterScatterPlot(embeded,pred,'t-SNE Covid-19 - Clustered(K-Means) - Tf-idf with Plain Text')
    return embeded,kmeans,x    

In [0]:
research_dataframe = readExcelToDataFrame('/content/drive/My Drive/Colab Notebooks/kaggle_covid_19.csv')
# clean_data =dataCleaningProcess(research_dataframe)
research_dataframe.head()

,doc_id,source,title,abstract,text_body
0,NaN,NaN,NaN,NaN,NaN
1,850dc8add6efb76a6dc341cb7a5a236c1dd3da7c,BIORXIV,Expanded skin virome in DOCK8-deficient patients,NaN,"recurrent cutaneous and systemic infections, a..."
2,dd6c1a719bd75ba7fb7c62291a676b39c99bb0bc,BIORXIV,Veterinary Science Molecular characterization ...,The cDNA nucleotide sequence of genome segment...,Members of the family Birnaviridae have 2-segm...
3,e88710cd7b238794bd75f21d308bc5442ad9713f,BIORXIV,NaN,NaN,The common cold is one of the most prevalent a...
4,d46d2f01a570bfb6e3b2e65ec50972d1649b6a3d,BIORXIV,Severe Acute Respiratory Syndrome: Lessons fro...,An outbreak of severe acute respiratory syndro...,A n outbreak of severe acute respiratory syndr...


In [0]:
research_dataframe.shape

(13203, 5)

In [0]:
meta_data = readExcelToDataFrame('/content/drive/My Drive/Colab Notebooks/metadata.csv')

In [0]:
meta_data.head(1)

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,PMC,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,PMC140314,12525263.0,no-cc,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",BMC Public Health,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...


In [0]:
meta_data_filter = meta_data.query('doi == "10.1186/1471-2458-3-5"')
meta_data_filter.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,PMC,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,PMC140314,12525263.0,no-cc,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",BMC Public Health,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...


In [0]:
# doi,publish_time,authors,url,title,inference
meta_data_filter = meta_data.query('doi == "10.5582/ddt.2011.v5.5.220"')
meta_data_filter.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
22476,2vb63yn6,NaN,PMC,A sensitive near-infrared fluorescent probe fo...,10.5582/ddt.2011.v5.5.220,PMC3265084,22282719.0,unk,Near-infrared (NIR) absorbing dyes represent a...,2011-01-01,"Luan, Yepeng; Yang, Qiuhong; Xie, Yumei; Duan,...",Drug Discoveries & Therapeutics,NaN,NaN,False,False,NaN,https://www.jstage.jst.go.jp/article/ddt/5/5/5...


**Data Frame Helper Functions for search Context**

In [0]:
def prepare_search_data(_meta_data_frame,research_dataframe):
  _meta_data_frame["doc_id"] = _meta_data_frame["sha"]
  _meta_data_frame = cleanEmptyData('doc_id', _meta_data_frame)
  _meta_data_frame = cleanEmptyData('publish_time', _meta_data_frame)
  meta_data_filter = _meta_data_frame[_meta_data_frame['publish_time'].str.contains('2019') | _meta_data_frame['publish_time'].str.contains('2020')]  
  research_dataframe_clean = cleanEmptyData('doc_id', research_dataframe)
  research_dataframe_clean = cleanEmptyData('text_body', research_dataframe_clean)
  tmp_data_frame  = research_dataframe_clean.merge(meta_data_filter, on='doc_id', how='right')
  clean_process_data = tmp_data_frame.drop(["source", "abstract_x", "cord_uid", "abstract_x","sha","source_x","title_y","pmcid","pubmed_id","license","abstract_y","journal","Microsoft Academic Paper ID","WHO #Covidence","has_pdf_parse","has_pmc_xml_parse","full_text_file"], axis=1)
  clean_process_data = cleanEmptyData('text_body', clean_process_data)
  clean_process_data = clean_process_data.rename(columns={'title_x': 'title'})
  clean_process_data = clean_process_data.query('doi == "10.5582/ddt.2011.v5.5.220"')
  columns = ["doc_id","doi", "publish_time", "authors","url","title", "text_body"]
  # doi,publish_time,authors,url,title,inference
  clean_process_data = clean_process_data.reindex(columns=columns)
  return clean_process_data

In [0]:
search_data = prepare_search_data(meta_data,research_dataframe)
search_data.head()

,doc_id,doi,publish_time,authors,url,title,text_body


In [0]:
nlp.max_length = 10300000000000

In [0]:
def search_text(raw_text, keywords):
  result = []
  doc = nlp(raw_text)
  for sent in doc.sents:
    text = sent.text
    no_match_was_found = re.search(keywords, text, flags=re.IGNORECASE)
    if no_match_was_found:    
      result.append(text) 
  
  #if(len(result)>0):
    #print(result)      
  return result


In [0]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [0]:
def search_inference_keys(text, keywords):
  sentences = split_into_sentences(text)
  txt = ''
  for sent in sentences:
    r = re.compile(keywords,flags=re.IGNORECASE)
    if len(r.findall(sent))>0:         
      txt = str(txt) + str(sent)
  return txt

In [0]:
def check_exist_multiple_keywords(text, keywords):
  r = re.compile(keywords, flags=re.IGNORECASE)
  if len(r.findall(text))>0:    
    return True
  else:
    return False

In [0]:
search_data.isnull()

,doc_id,doi,publish_time,authors,url,title,text_body
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...
2282,False,False,False,False,False,False,False
2283,False,False,False,False,False,False,False
2284,False,True,False,False,False,False,False
2285,False,False,False,False,False,False,False


In [0]:
def searc_by_keys_as_excel(keyword, src_data_frame):
  data_frame = src_data_frame
  data_frame['search_key_status'] =data_frame.loc[:,'text_body'].apply(lambda x: check_exist_multiple_keywords(x,keyword))
  print('Completed Search Indexing')
  process_data_frame = data_frame.query('search_key_status == True')
  ## filter on corona and covid 19 related data
  process_data_frame.loc[:,'inference'] = process_data_frame.loc[:,'text_body'].apply( lambda x: search_inference_keys(x,keyword))
  print('Completed Inference Search')
  #process_data_frame = process_data_frame.query('inference != "failed"')
  final_data = process_data_frame.drop(["search_key_status","text_body"], axis=1)
  final_data.head()
  final_data.to_excel(str(keyword) + '_result.xlsx', sheet_name='keyword')
  print('Completed')
  return final_data

In [0]:
def searc_covid_by_keys_as_excel(keyword, src_data_frame):
  data_frame = src_data_frame
  data_frame['search_key_status'] =data_frame.loc[:,'text_body'].apply(lambda x: check_exist_multiple_keywords(x,keyword))
  print('Completed Search Indexing')
  process_data_frame = data_frame.query('search_key_status == True')

  process_data_frame['search_covid_content'] =process_data_frame.loc[:,'text_body'].apply(lambda x: check_exist_multiple_keywords(x,'covid-19|sars-cov-2|2019-ncov|ncov-19|coronavirus'))
  process_data_frame = process_data_frame.query('search_covid_content == True')
  ## filter on corona and covid 19 related data
  process_data_frame.loc[:,'inference'] = process_data_frame.loc[:,'text_body'].apply( lambda x: search_inference_keys(x,keyword))
  print('Completed Inference Search')
  #process_data_frame = process_data_frame.query('inference != "failed"')
  final_data = process_data_frame.drop(["search_key_status","text_body"], axis=1)
  final_data.head()
  final_data.to_excel(str(keyword) + '_result.xlsx', sheet_name='keyword')
  print('Completed')
  return final_data

In [0]:
final_data =searc_covid_by_keys_as_excel('remdesivir|lopinavir|ritonavir|oseltamivir|favipiravir|sofosbuvir|corticosteroids',search_data)
final_data.head()

Completed Search Indexing


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

Completed Inference Search
Completed


,doc_id,doi,publish_time,authors,url,title,search_covid_content,inference
0,43d2d4072d40a3964ea330342b9222376d700531,10.3201/eid2509.181937,2019-09-10,"Jing, Shuping; Zhang, Jing; Cao, Mengchan; Liu...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,Lassa fever in travelers from West Africa,True,After antiinflammatory and antiviral treatment...
54,5d4f1f02d0e731966dddd635e8fa7bfdc169d3b9,10.1177/2333794x19845074,2019-05-01,"Smith, Ryan Kenneth; Gerrits, Peter M.",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,Case Report A Rare Case of Autoimmune Polyglan...,True,When initiating hormone therapy for treatment ...
55,afa4ca223097866f6e31d75570448a22db0f72c5,10.2147/idr.s231087,2019-11-07,"AlRuthia, Yazed; Somily, Ali M; Alkhamali, Ama...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,Estimation Of Direct Medical Costs Of Middle E...,True,75%) as well as those on mechanical ventilatio...
69,f3fa89e819a64d2d6c74f6d590661ca435878c26,10.1097/jcma.0000000000000270,2020,"Wu, Yi-Chi; Chen, Ching-Sung; Chan, Yu-Jiun",https://doi.org/10.1097/jcma.0000000000000270,The outbreak of COVID-19: An overview,True,Remdesivir: The experimental drug is a novel n...
100,8cfe567d42fa459cb4364e240ddfe29629d03f85,10.5114/ceji.2019.92790,2020-01-20,"WARDZYŃSKA, ALEKSANDRA; PAWEŁCZYK, MAŁGORZATA;...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,The profile of respiratory pathogens in induce...,True,None of the individuals demonstrated any sympt...


In [0]:
search_data.isnull()

,doc_id,doi,publish_time,authors,url,title,text_body,search_key_status
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...
2282,False,False,False,False,False,False,False,False
2283,False,False,False,False,False,False,False,False
2284,False,True,False,False,False,False,False,False
2285,False,False,False,False,False,False,False,False


In [0]:
check_exist_multiple_keywords('covid-19','covid-19|sars-cov-2|2019-ncov|ncov-19|coronavirus')

True

In [0]:
final_data.isnull()

,doc_id,doi,publish_time,authors,url,title,inference
55,False,False,False,False,False,False,False
79,False,False,False,False,False,False,False
112,False,False,False,False,False,True,False
113,False,False,False,False,False,False,False
128,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
2094,False,False,False,False,False,False,False
2099,False,False,False,False,False,False,False
2127,False,False,False,False,False,False,False
2155,False,False,False,False,False,False,False


In [0]:
 process_data_frame['corona_filter'] =process_data_frame.loc[:,'text_body'].apply(lambda x: check_exist_multiple_keywords(x,'covid-19|sars-cov-2|2019-ncov|ncov-19|coronavirus'))

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Found GPU at: /device:GPU:0


In [0]:
s = "These are roses and lilies and orchids, but not marigolds or .."
k = 'ROses|not'
r = re.compile(k)
print(len(r.findall(s)))

1


In [0]:
def search_inference_test(text, keywords):
  total_keyword = len(keywords.split("|"))
  print('total Keword search :' , total_keyword)
  sentences = split_into_sentences(text)
  txt = ''
  for sent in sentences:
    #match_was_found = re.search(keywords, sent, flags=re.IGNORECASE)
    r = re.compile(keywords, flags=re.I | re.X)
    print('Search Result:', len(r.findall(sent)))
    if len(r.findall(sent)) == total_keyword:         
      txt = str(txt) + str(sent)
  return txt

In [0]:
s ="Operational issues associated with wearing disposable facemasks to maximise their preventive effectiveness should be also publicised and educated to the general public including correct ways of wearing facemask, hygiene practices across the procedure of mask wearing, disposal of used masks."
k = 'issues associated'
text_result = search_inference_keys(s,k)
print(text_result)

[]
0 -- 1
failed


In [0]:
import re
ke = 'Adrenal insufficiency'
s = "Adrenal insufficiency is decreased or deficient production of glucocorticoids and/or mineralocorticoids that has potential to be life-threatening.1,2 First described by Thomas Addison in 1855, this disease can present insidiously with weakness, fatigue, weight loss, abdominal pain, nausea, vomiting, hypotension, salt cravings, depression, anxiety, as well as mucosal and epithelial hyperpigmentation.1, 2 Adrenal insufficiency is related to dysfunction of the hypothalamic-pituitary-adrenal axis.1, 2 Primary disease refers to pathology of the adrenal cortex; secondary refers to inadequate or deficient release of adrenocorticotropic hormone (ACTH) or inappropriate response to this hormone by the adrenal gland; and tertiary refers to disordered hypothalamic production of corticotropin releasing hormone.1 Primary adrenal disease occurs more"
r = re.compile('b(?:insufficiency|deficient)\b', flags=re.IGNORECASE)
result = r.findall(s);
print(result)

[]
